In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import shutil, zipfile

filename = 'dataset_intrusos.zip'
shutil.copy("drive/My Drive/Material/" + filename, "./" + filename)
zip_ref = zipfile.ZipFile(filename, 'r')
zip_ref.extractall()
zip_ref.close()

filename = 'detector.zip'
shutil.copy("drive/My Drive/Material/" + filename, "./" + filename)
zip_ref = zipfile.ZipFile(filename, 'r')
zip_ref.extractall()
zip_ref.close()


In [3]:
!pip install numpy==1.16.2
import numpy as np
print(np.__version__)

     |████████████████████████████████| 17.3MB 3.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


1.16.4


In [6]:
from detector.yolo import YOLO
import moviepy.editor as mpy
import time
import numpy as np
yolo = YOLO(model_path = 'yolotiny_01.h5' ,  #'detector/yolo-tiny.h5' 'yolotiny_01.h5' 'ep009-loss8.143-val_loss7.033.h5'
        anchors_path = 'detector/tiny_yolo_anchors.txt',
        classes_path = 'detector/coco_classes.txt',
        score= 0.05, 
        iou = 0.45,
        model_image_size = (416, 416))

yolotiny_01.h5 model, anchors, and classes loaded.


In [7]:
from keras.preprocessing import image

folder = 'dataset_intrusos/'
true_detections = np.load(folder + 'true_detections_intrusos.npy').item()

k = 0
yolov3_detections = {}
for filename in true_detections: 
    img = image.load_img(folder + filename)
    dimg, out_boxes, out_scores, out_classes = yolo.detect_image(img)
    index_vehicles = np.where(np.array(out_classes) == 0)[0]  #detectar personas
    final_detections = []
    for i in index_vehicles:       
        top, left, bottom, right = out_boxes[i]
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(img.size[1], np.floor(bottom + 0.5).astype('int32'))
        right = min(img.size[0], np.floor(right + 0.5).astype('int32'))
        final_detections.append(((left, top), (right, bottom)))

    yolov3_detections[filename] = final_detections

    if k%50==0:
        print('Imagen numero', k, 'fue procesada')
    k+=1

print('Diccionario con las listas de detecciones por imagen obtenidas por YOLOV3: ')   
yolov3_detections

Imagen numero 0 fue procesada
Imagen numero 50 fue procesada
Imagen numero 100 fue procesada
Imagen numero 150 fue procesada
Imagen numero 200 fue procesada
Imagen numero 250 fue procesada
Imagen numero 300 fue procesada
Diccionario con las listas de detecciones por imagen obtenidas por YOLOV3: 


{'aguacateMi0.jpg': [],
 'aguacateMi1.jpg': [],
 'aguacateMi10.jpg': [],
 'aguacateMi100.jpg': [((224, 210), (255, 276)), ((274, 142), (300, 203))],
 'aguacateMi101.jpg': [((348, 157), (374, 216)), ((294, 228), (322, 292))],
 'aguacateMi102.jpg': [((357, 220), (406, 301))],
 'aguacateMi103.jpg': [((384, 185), (421, 266))],
 'aguacateMi104.jpg': [((407, 164), (437, 227)), ((460, 247), (502, 305))],
 'aguacateMi105.jpg': [((441, 154), (470, 206)), ((510, 322), (538, 375))],
 'aguacateMi106.jpg': [],
 'aguacateMi107.jpg': [((393, 136), (427, 201)), ((350, 398), (405, 456))],
 'aguacateMi108.jpg': [((375, 164), (400, 223))],
 'aguacateMi109.jpg': [((182, 242), (235, 325)), ((345, 161), (372, 243))],
 'aguacateMi11.jpg': [],
 'aguacateMi110.jpg': [((129, 205), (158, 259)),
  ((294, 178), (321, 243)),
  ((139, 213), (173, 275))],
 'aguacateMi111.jpg': [((259, 226), (293, 293)), ((116, 169), (153, 233))],
 'aguacateMi112.jpg': [((258, 272), (294, 343)),
  ((147, 151), (173, 205)),
  ((274, 27

In [8]:
import cv2
from cv2 import imread
import matplotlib.pyplot as plt
import random

 # Visualizacion para verificar
plt.figure(figsize=(25,17))
for i in range(25):
    filename = 'aguacateMi' + str(random.randint(0,250)) + '.jpg'
    img = imread(folder + filename)
    true_boxes = true_detections[filename]   
    for box in true_boxes:
        cv2.rectangle(img, box[0], box[1], (255,0,0), 6)
    boxes = yolov3_detections[filename]   
    for box in boxes:
        cv2.rectangle(img, box[0], box[1], (0,0,255), 6)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(5,5,i+1)
    plt.imshow(img) 
    plt.axis('off') 
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [9]:
from detector.metrics import f1score_evaluation

P, R, F1 = f1score_evaluation(true_detections, yolov3_detections)
print('precision =', P)
print('sensibilidad =', R)
print('f1-score =', F1)

precision = 0.6730784610180385
sensibilidad = 0.5170903042035
f1-score = 0.584862174728399


In [0]:
"yolo-tiny"
score= 0.05, 
iou = 0.45,
precision = 0.6182
sensibilidad = 0.3091
f1-score = 0.4122

"ep006-loss9.073-val_loss7.261.h5"
score = 0.05
iou = 0.45
precision = 0.6415
sensibilidad = 0.4423
f1-score = 0.5236

'yolotiny_01.h5'
score = 0.05
iou = 0.45
precision = 0.6731
sensibilidad = 0.5171
f1-score = 0.5849